
## Without Zeeman and with orbital effects, spin-orbit has little effect on $I_c$

## With spin-orbit and with orbital effects, Zeeman has little effect on $I_c$

## Without orbital effects and without spin-orbit, the $B$ dependence scales both with $L$ and $\mu$

## In single mode regime, Zeeman has the predominant effect on $I_c$

## Both in the single mode and multimode regime, the orbital effect has the predominant effect on $I_c$

## $0-\pi$ transitions?
* $L=640, \mu=50$
* $L=640, \mu=40$
* $L=160, \mu=30$


In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt

import numpy as np
import pandas as pd
import holoviews as hv
# hv.notebook_extension()
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
          'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 300
         }

plt.rcParams.update(params)
plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

import matplotlib.colors as colors
    
class HistogramNormalize(colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                       len(self.sorted_data))
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm

    
def open_mtx(fname):
    with open(fname, 'rb') as f:
        line1 = f.readline()
        *shape, entry_size = [int(i) for i in  f.readline().split()]
        print('shape: ', shape)
        data = np.fromfile(f, (np.float64 if entry_size == 8 else np.float32))
        data = data.reshape(shape).squeeze()
    
    keys = ['units', 'plot_settings', 'xname', 'xmin', 'xmax', 
            'yname', 'ymin', 'ymax', 'zname', 'zmin', 'zmax']
    values = line1.decode("utf-8").replace('\n', '').split(',')
    meta_data = dict(zip(keys, values))
    meta_data = {}
    for key, val in zip(keys, values):
        try:
            meta_data[key] = float(val)
        except:
            meta_data[key] = val
    return meta_data, data

# same but with disorder

In [ ]:
df['disorder'].unique()

In [ ]:
fig, axs = plt.subplots(4, 1, sharey=False, sharex=True, figsize=(fig_width, 1.5*fig_height), 
                        gridspec_kw={'height_ratios':[2, 1, 2, 1]})

plt.subplots_adjust(bottom=0.2, left=0.125, right=0.90, top=0.9, hspace=0.25, wspace=0.15)
current = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: (
    gb.get_group((orbital, g, alpha, mu, disorder, 0))['current_c'].values * 2.09757)
# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin

phase = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0: gb.get_group((orbital, g, alpha, mu, disorder, 0))['phase_c'].values
B_x = total['B_x']

def plot_curves(ax, mu, plot=current):
    args = {'lw': 0.6, 'alpha': 1}
    if mu == 10:
        disorder=70
    else:
        disorder=110
    ax.plot(B_x, plot(disorder=disorder, mu=mu), label='all effects', c='b', **args)
    ax.plot(B_x, plot(disorder=disorder, orbital=False, mu=mu), label=r'$\bm{A}=0$', c='g', **args)
    ax.plot(B_x, plot(disorder=disorder, g=0, mu=mu)*1.025, label='$g=0$', c='r', **args) # small off-set to make the curves not overlap
    ax.plot(B_x, plot(disorder=disorder, alpha=0, mu=mu), label=r'$\alpha=0$', c='m', **args)
    ax.plot(B_x, plot(disorder=disorder, orbital=False, alpha=0, mu=mu), label=r'$\alpha=0, \; \bm{A}=0$', c='c', **args)
    ax.plot(B_x, plot(disorder=disorder, alpha=0, g=0, mu=mu), label=r'$\alpha=0, \; g=0$', c='y', **args)
#     ax.plot(B_x, plot(mu=mu, disorder=70), label=r'disorder, 70 meV', c='k', **args)
#     ax.plot(B_x, plot(mu=mu, disorder=100), label=r'disorder, 100 meV', c='k', ls=':', **args)
    return ax

ymax = 0.1
axs[0] = plot_curves(axs[0], 10)
# axs[0].set_yticks([0, 0.2, 0.4, 0.6])
axs[0].set_ylim(0, ymax)
axs[0].set_xlabel('')
axs[0].set_ylabel('$I_c$ [nA]')
leg0 = axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.9), ncol=2, prop={'size': 6})

axs[0].text(0.01, 0.8*ymax, '(a)')

ymax = 6
axs[2] = plot_curves(axs[2], 50)
# axs[2].set_yticks([0, 20, 40, 60, 80])
axs[2].set_ylim(0, ymax)
axs[2].set_ylabel('$I_c$ [nA]')
axs[2].text(0.01, 0.8*ymax, '(c)')

for i in argrelextrema(current(orbital=False, alpha=0, mu=10, disorder=70), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[0].plot((x, x), (y1, y2), 'c--')

for i in argrelextrema(current(orbital=False, alpha=0, mu=50, disorder=110), np.less):
    x = B_x.iloc[i]
    y1, y2 = 0, ymax
    axs[2].plot((x, x), (y1, y2), 'c--')
    
def first_min_max(arr):
    return B_x.iloc[argrelextrema(arr, np.less)[0]], B_x.iloc[argrelextrema(arr, np.greater)[0]]

B1, B2 = first_min_max(current(orbital=False, alpha=0, mu=10)) # for next cell
B3, B4 = first_min_max(current(orbital=False, alpha=0, mu=50)) # for next cell

axs[1] = plot_curves(axs[1], 10, phase)
axs[1].set_yticks([-3.14, 0, 3.14])
axs[1].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[1].set_ylabel(r'$\theta_c$')
axs[1].text(0.01, -2.15, '(b)')

axs[3] = plot_curves(axs[3], 50, phase)
axs[3].set_yticks([-3.14, 0, 3.14])
axs[3].set_yticklabels(['$-\pi$', 0, '$\pi$'])
axs[3].set_xlabel('$B_x$ [$T$]')
axs[3].set_ylabel(r'$\theta_c$')
axs[3].text(0.01, -2.1, '(d)')
fig.savefig('figures/critical_currents.pdf', format='pdf', dpi=300, bbox_inches="tight", additional_artists=[leg0])
plt.show()

# Gate dependence

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.subplot.left': 0.1})

In [ ]:
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x__A_in_SM_and_SC.hdf')
gb = df.groupby(['gate_size'])
gr = gb.get_group((640))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['Vs'], gr.max()['Vs'])
norm = HistogramNormalize(np.array(gr['current_c']))
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', vmax=20, 
              norm=norm, origin='lower')
im = ax.imshow(gr.pivot(index='B_x', columns='Vs', values='current_c').T, **kwargs)
ax.set_xlabel('Magnetic field $B_x$ [$T$]')
ax.set_ylabel(r'$V_\textrm{gate}$ [meV]')
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [meV$/\Phi_0$]', ticks=cbar_ticks)
cb.ax.set_yticklabels(cbar_ticks)
fig.savefig('figures/gate_dependence.pdf', format='pdf', dpi=300)

In [ ]:
df['disorder'].unique(), df['mu'].unique()

In [ ]:
df = pd.read_hdf('data/current_as_function_of_gate_and_B_x_with_disorder_combined.hdf')
gb = df.groupby(['disorder', 'gate_size'])
gr = gb.get_group((70, 80))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['V'], gr.max()['V'])

data = gr.pivot(index='B_x', columns='V', values='current_c').T * 2.09757

norm = HistogramNormalize(np.array(data), vmin=0, vmax=2)
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
#               norm=norm, 
#               vmax=2,
              origin='lower')
im = ax.imshow(data, **kwargs)

# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin
ax.set_xlabel('Magnetic field $B_x$ [$T$]')
ax.set_ylabel(r'$V_\textrm{gate}$ [meV]')
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [nA]')
# cb.set_ticks(cbar_ticks)
fig.savefig('figures/gate_dependence.pdf', format='pdf', dpi=300)

In [ ]:
%%opts Image [aspect='square']
import funcs
def holomap_from_df(fname, x, y, z=None, drops=[], xlims=(None, None), ylims=(None, None)):
    df = pd.read_hdf(fname)
    udf = df.drop([x, y, *drops], axis=1)
    if z is not None:
        udf = udf.drop([z], axis=1)
    udf = funcs.drop_constant_columns(udf)
    dims = [hv.Dimension(col, values=udf[col]) for col in udf.columns]
    gb = df.groupby(udf.columns.tolist())
    hm = {}
    for i, gr in gb:
        table = gr.pivot_table(index=x, columns=y, values=z)
        extents = gr[x].min(), gr[y].min(), gr[x].max(), gr[y].max()
        hm[i] = hv.Image(np.rot90(table), kdims=[x, y], bounds=extents)
        
    return hv.HoloMap(hm, kdims=udf.columns.tolist())


hm = holomap_from_df('data/current_as_function_of_gate_and_B_x_with_disorder_combined.hdf', 
                   x='B_x', y='V', z='current_c', drops=('currents', 'phases', 'phase_c', 'git_hash'))

print(hm.items())

hm[640, 100].select(B_x=(0, 0.5), V=(0, 10)).data

In [ ]:
df = pd.read_hdf('data/current_as_function_of_gate_and_B_x_with_disorder_combined.hdf')
gb = df.groupby(['disorder', 'gate_size'])
gr = gb.get_group((100, 640))
gr = gr[gr['B_x'] < 0.5]
gr = gr[gr['V'] >= 15]
gr = gr[gr['V'] <= 25]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (gr.min()['B_x'], gr.max()['B_x'], gr.min()['V'], gr.max()['V'])
data = gr.pivot(index='B_x', columns='V', values='current_c').T * 2.09757
norm = HistogramNormalize(np.array(data), vmax=1) 
norm = colors.PowerNorm(gamma=0.5)
kwargs = dict(extent=extent, cmap='viridis', aspect='auto', 
              norm=norm, 
              interpolation='nearest',
#               vmax=10,
              origin='lower')
im = ax.imshow(data, **kwargs)

# multiplied by 2e/hbar * k_B * T
# http://www.wolframalpha.com/input/?i=(2+*+electronic+charge+%2F+hbar+)+*+boltzmann+constant+*+50e-3+kelvin
ax.set_xlabel('Magnetic field $B_x$ [$T$]')
ax.set_ylabel(r'$V_\textrm{gate}$ [meV]')
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(im, cax=cax, 
#                   extend='max', 
                  label=r'$I_c$ [nA]')
# cb.set_ticks(cbar_ticks)
fig.savefig('figures/gate_dependence.pdf', format='pdf', dpi=300)


In [ ]:
df['disorder'].unique(), df['mu'].unique(), df['gate_size'].unique()

# Rotating field

In [ ]:
df = pd.read_hdf('data/rotation_of_field__L_640_nm__infinite_leads.hdf')
df['theta'] = (df['theta'] * 180 / np.pi).round(0).astype(int)
gb = df.groupby(['theta', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

theta_0 = gb.get_group((0, 1, 640, True, True, 50, 20, 50))
theta_30 = gb.get_group((30, 1, 640, True, True, 50, 20, 50))
theta_60 = gb.get_group((60, 1, 640, True, True, 50, 20, 50))
theta_90 = gb.get_group((90, 1, 640, True, True, 50, 20, 50))

B_x = theta_0['B']

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125, 0.2, (0.95-0.125), (0.95-0.2)])
ax.plot(B_x, theta_0['current_c'], label=r'$\bm{B} \parallel \hat{x}$')
ax.plot(B_x, theta_90['current_c'], label=r'$\bm{B} \parallel \hat{z}$')
ax.plot(B_x, theta_30['current_c'], label=r'$\bm{B} \parallel \sqrt{3} \hat{x} + \hat{z}$')
ax.plot(B_x, theta_60['current_c'], label=r'$\bm{B} \parallel \hat{x} + \sqrt{3} \hat{z}$')
ax.set_yticks(np.arange(0, 100, 10))
ax.set_ylim(0, 40)
ax.set_xlabel(r'Magnetic field $|\bm{B}|$ [$T$]')
ax.set_ylabel('$I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=1)
fig.savefig('figures/rotation_of_field.pdf', format='pdf', dpi=300)
plt.show()

$U H(B, \phi) U^\dagger = H(B, -\phi), \quad U = \sigma_x \delta(x+x')$

# Current as function of $\mu$ and $B_x$

In [ ]:
df = pd.read_hdf('data/no_disorder_B_vs_I_c__L_640_nm__infinite_leads__varying_mu.hdf')
gb = df.groupby(['orbital', 'g', 'alpha'])

gr = gb.get_group((True, 50, 20))
for (orbital, g, alpha), gr in gb:
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
    extent = (gr.B_x.min(), gr.B_x.max(), gr.mu.min(), gr.mu.max())
    norm = HistogramNormalize(gr.current_c.values)
    kwargs = dict(extent=extent, cmap='viridis', aspect='auto', vmax=20, 
                  norm=norm, origin='lower')
    im = ax.imshow(gr.pivot_table(index='B_x', columns='mu', values='current_c').T, **kwargs)
    ax.set_xlabel('Magnetic field $B_x$ [$T$]')
    ax.set_ylabel(r'$\mu$ [meV]')
    ax.set_title(r'$\bm{{A}} {}0, \; g={}, \; \alpha={}$'.format((r'\ne' if orbital else '='), g, alpha))
    cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
    cbar_ticks = [0, 5, 10, 15, 20, 25]
    cb = fig.colorbar(im, cax=cax, extend='max', label=r'$I_c$ [meV$/\Phi_0$]', ticks=cbar_ticks)
    cb.ax.set_yticklabels(cbar_ticks)
    plt.show()


# $I_c(\mu, \alpha)$ for a 1D wire

In [ ]:
df = pd.read_hdf('data/1d_alpha_vs_B_x.hdf')
gb = df.groupby('mu')

fig, axs = plt.subplots(2, 3, sharey=True, sharex=True, figsize=(fig_width, fig_height))
plt.subplots_adjust(bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.15, wspace=0.15)

for i, (ax, (mu, gr)) in enumerate(zip(axs.flatten(), gb)):
    norm = HistogramNormalize(gr.current_c.values)
    extent = (gr.B_x.min(), gr.B_x.max(), gr.alpha.min(), gr.alpha.max())
    ax.imshow(gr.pivot_table(index='B_x', columns='alpha', values='current_c').T.values, 
              origin='lower', cmap='viridis', extent=extent, aspect='auto', norm=norm)

    ax.set_yticks([0, 50, 100])
    ax.set_xticks([0, 0.7, 1.4])
    ax.set_xticklabels(['0', '0.7', '1.4'])

    if i in [3, 4, 5]:
        ax.set_xlabel('$B_x$ [T]')
    if i in [0, 3]:
        ax.set_ylabel(r'$\alpha$ [meV$\cdot$nm]')

    ax.text(0.08, 80, '({})'.format('abcdef'[i]))
fig.savefig('figures/currents_1D_alpha_vs_B_x.pdf', format='pdf', dpi=300)